In [1]:
# When do the measures get off
missing_nodes = [0.1, 0.3, 0.5, 0.7]  # Percent of missing nodes

In [2]:
import networkx as nx
import rumor_centrality.rumor_detection as raw
import rumor_centrality.jordan_center_alternative as jo
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from rumor_centrality.graph_generator import scale_free, small_world
from time import time
from multiprocessing import Pool
from typing import List, Tuple
from rumor_centrality.graph_simulations import si
import random

In [3]:
samples = 5
sizes = [100, 200, 300, 400, 500, 1000]

In [4]:
def get_gnp_01(n):
        return nx.fast_gnp_random_graph(n, 0.1)
def get_gnp_03(n):
    return nx.fast_gnp_random_graph(n, 0.3)
def get_gnp_06(n):
    return nx.fast_gnp_random_graph(n, 0.6)

In [5]:
def get_rand_tree(n):
    return nx.random_tree(n)

In [6]:
def get_complete_graph(n):
    return nx.complete_graph(n)

In [7]:
def get_timing(g) -> Tuple[List[int], int, float]:
    start = time()
    centers = raw.get_center_prediction(raw.networkx_graph_to_adj_list(g), use_fact=False)
    return centers, len(g.edges), time() - start

In [8]:
def gather_results(graph_generating_callback, missing: float):
    percent_of_infection = 0.2
    
    results = []
    with Pool(samples) as p:
        for n in tqdm(sizes):
            g = graph_generating_callback(n)
            g_read_only = g.copy(as_view=True)

            infected_g, real_centers = si(g, -1, 0.1, 1, int(percent_of_infection * n))
            
            missing_nodes_count = int(len(infected_g.nodes) * missing)
            new_graph = None
            while new_graph is None or nx.is_connected(new_graph) is False:
                new_graph = infected_g.copy()
                nodes_to_remove = random.sample(new_graph.nodes, missing_nodes_count)
                for node_to_remove in nodes_to_remove:
                    new_graph.remove_node(node_to_remove)

            data = p.map(get_timing, [new_graph for _ in range(samples)])
            
            hop_distances = []
            timings = []
            edge_counts = []
            for d in data:
                centers = d[0]
                edges = d[1]
                edge_counts.append(edges)
                timing = d[2]
                timings.append(timing)
                for center in centers:
                    for r_c in real_centers:
                        hop_distance = nx.shortest_path_length(g_read_only, r_c, center)
                        hop_distances.append(hop_distance)

            r = {
                "n": n,
                "m": sum(edge_counts) / len(edge_counts),
                "avg time": sum(timings) / len(timings),
                "degree": sum(edge_counts) / len(edge_counts) / n,
                "hop_distances": hop_distances,
                "missing": missing,
            }
            results.append(r)
    return results

In [9]:
experiments = {
    # "GNP 0.1": get_gnp_01,
    # "GNP 0.3": get_gnp_03,
    "GNP 0.6": get_gnp_06,
    # "Rand Tree": get_rand_tree,
    # "Compl. Graph": get_complete_graph,
}

results = {}
for missing_percent in missing_nodes:
    r = {}
    for key, callback in experiments.items():
        r[key] = gather_results(callback, missing_percent)
    results[missing_percent] = r

100%|███████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.67it/s]


In [11]:
for key, vals in results.items():
    print(f"Missing: {key}")
    for k, v in vals.items():
        print(k)
        for _v in v:
            print(_v)
    print()

Missing: 0.1
GNP 0.6
{'n': 100, 'm': 621.0, 'avg time': 0.0067158699035644535, 'degree': 6.21, 'hop_distances': [1, 1, 1, 1, 1], 'missing': 0.1}
{'n': 200, 'm': 1988.0, 'avg time': 0.03604316711425781, 'degree': 9.94, 'hop_distances': [1, 1, 1, 1, 1], 'missing': 0.1}
{'n': 300, 'm': 13339.0, 'avg time': 0.4562243461608887, 'degree': 44.46333333333333, 'hop_distances': [2, 2, 2, 2, 2], 'missing': 0.1}
{'n': 400, 'm': 29741.0, 'avg time': 1.4837453842163086, 'degree': 74.3525, 'hop_distances': [2, 2, 2, 2, 2], 'missing': 0.1}
{'n': 500, 'm': 45648.0, 'avg time': 2.9435638904571535, 'degree': 91.296, 'hop_distances': [2, 2, 2, 2, 2], 'missing': 0.1}
{'n': 1000, 'm': 228480.0, 'avg time': 37.2739857673645, 'degree': 228.48, 'hop_distances': [2, 2, 2, 2, 2], 'missing': 0.1}

Missing: 0.3
GNP 0.6
{'n': 100, 'm': 175.0, 'avg time': 0.001974344253540039, 'degree': 1.75, 'hop_distances': [2, 2, 2, 2, 2], 'missing': 0.3}
{'n': 200, 'm': 2155.0, 'avg time': 0.035749292373657225, 'degree': 10.775,